# Oğuzhan Kır
# 528231089

# Problem definition

You are trying to find the best parking space to use that minimizes the time needed to get to your
restaurant. There are 50 parking spaces, and you see spaces 1,2, . . . ,50 in order. As you approach each
parking space, you see whether it is full or empty. We assume, somewhat heroically, that the probability
that each space is occupied follows an independent Bernoulli process, which is to say that each space will
be occupied with probability p, but will be free with probability 1 - p, and that each outcome is
independent of the other.
It takes 2 seconds to drive past each parking space and it takes 8 seconds to walk past. That is, if we park
in space n, it will require 8(50 - n) seconds to walk to the restaurant. Furthermore, it would have taken
you 2n seconds to get to this space. If you get to the last space without finding an opening, then you will
have to drive into a special lot down the block, adding 30 seconds to your trip.
We want to find an optimal strategy for accepting or rejecting a parking space.

- (a) Give the sets of state and action spaces.
- (b) Give the optimality equations for solving this problem.
- (c) You have just looked at space 45, which was empty. There are five more spaces remaining (46 through
50). What should you do? Using p = 0.6, find the optimal policy by solving your optimality equations for
parking spaces 46 through 50.
- d) Give the optimal value in part (c) corresponding to your optimal solution.

# States and action spaces

State Space: The state can be represented by the number of the current parking space, i.e., $s \in \{1, 2, \ldots, 52\}$, where 51 represents special lot, 52 represents terminal state.

Action Space: The action could be to park or move to the next parking space, i.e., $a \in \{P, C\}$, where P represents parking and C represents continuing to the next parking spot.


In [1]:
# Define state and action spaces
states = [i for i in range(1, 53)]
action_space = {"P", "C"}

# Optimality equations

### Generic Bellman Optimality Equation

The generic Bellman optimality equation for the value function $ V^*(s)$ of a state $ s $ is:

$ V^*(s) = \max_{a \in A(s)} \sum_{s'} P(s' \mid s, a) [R(s, a, s') + \gamma V^*(s')] $

where:
- $ V^*(s) $  is the value of the optimal policy from state $s$.
- $A(s)$ is the set of actions available in state $s$.
- $P(s' \mid s, a) $ is the probability of transitioning to state $ s' $ from state $ s $ under action $ a $.
- $ R(s, a, s')$ is the reward received after taking action $ a $ in state $ s $ and transitioning to state $s'$.
-$ \gamma $ is the discount factor.
  
### Specialized Bellman Optimality Equation for the Parking Problem

For the parking problem , the Bellman optimality equation can be specialized as follows:

1. For $ s \lt 50 $:
$ V^*(s) = \max \left( -2 + p \cdot V^*(s+1) + (1-p) \cdot V^*(52), -2 + V^*(s+1) \right) $

2. For $ s = 50 $:
$ V^*(50) = \max \left( -30, -2 + p \cdot V^*(51) + (1-p) \cdot V^*(52) \right) $

3. For $ s = 51$ and $s = 52 $:
$ V^*(s) = 0 $

where:
- $ p $ is the probability that a parking space is occupied.
- The term $-2$ represents the cost of driving past each parking space.
- The term $-30$ represents the cost of driving to the special parking lot if no space is found before space 50.
- $ V^*(s+1) $ represents the value of the next space.
- $ V^*(51) $ represents the value of the special parking lot.



### Function Explanation

#### `is_terminal(s)`
Checks if a state $s$ is terminal, which means it's the end of the parking lot. It returns True if $s$ is 51 or 52, and False otherwise.

#### `get_transition_probabilities()`
Defines the transition probabilities for each state-action pair. For each state $s$, the transition probabilities are:
- For each state $s$:
  - If $s < 50$ and $s \neq 45$:
    - If choosing to continue driving ("C"), there is a 100% chance of moving to the next space ($s+1$): $P(s' = s+1 | s, \text{"C"}) = 1$.
    - If choosing to park ("P"), there is a 100% chance of moving to space 52 (terminal): $P(s' = 52 | s, \text{"P"}) = 1$.
  - If $s = 50$:
    - If choosing to continue driving ("C"), there is a 100% chance of moving to the next space ($s+1$): $P(s' = s+1 | s=50, \text{"C"}) = 1$.
    - If choosing to park ("P"), there is a probability $p$ of moving to space 51 (special lot) and a probability of $1-p$ of moving to space 52 (terminal): $P(s' = 51 | s=50, \text{"P"}) = p$, $P(s' = 52 | s=50, \text{"P"}) = 1-p$.
  - If $s = 45$:
    - If choosing to continue driving ("C"), there is a 100% chance of moving to the next space ($s+1$): $P(s' = s+1 | s=45, \text{"C"}) = 1$.
    - If choosing to park ("P"), there is a 100% chance of moving to space 52 (terminal): $P(s' = 52 | s=45, \text{"P"}) = 1$.





#### `get_rewards()`
Defines the rewards for each state-action pair. For each state $s$, the rewards are:

- If $s < 50$:
  - If choosing to continue driving ("C") or parking ("P"), there is a penalty of -2 for each step: $R(s, \text{"C"}, s+1) = R(s, \text{"P"}, s+1) = -2$.
- If parking at space 50, there is a penalty for parking and then walking to the restaurant but its equal to 0, and there is penalty for parking special lot:

$R(50, \text{"P"}, 52) = -8 \times (50 - 50) = 0$

$R(50, \text{"C"}, 51) = -30$
- If $s = 51$ or $s = 52$, there is no additional reward or penalty for parking: $R(51, \text{"P"}, 52) = R(52, \text{"P"}, 52) = 0$.

#### `policy_evaluation(policy)`
Evaluates the given policy to find the value function $V$. It iteratively updates the value of each state using the Bellman equation:

$ V(s) = \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma V(s') $

where $a$ is the action chosen according to the policy, $P(s' | s, a)$ is the transition probability, $R(s, a, s')$ is the reward, $V(s')$ is the value of the next state, and $\gamma$ is the discount factor.

#### `run_policy_iteration()`
Runs the policy iteration algorithm to find the optimal policy. It iteratively evaluates the policy, updates the policy based on the current value function $V$, and checks for policy convergence.


In [2]:
import numpy as np

# Probability that each space is occupied
p = 0.6

def is_terminal(s):
    """
    Check if a state is terminal (end of the parking lot).
    """
    return s in [51, 52]

# Define transition probabilities for each state-action pair
def get_transition_probabilities():
    """
    Define transition probabilities for each state-action pair.
    """
    transitions = {}
    for s in states:
        if s <= 50:
            # If space is less than or equal to 50, there is a 100% chance of moving to the next space
            transitions[(s, "C", s+1)] = 1
            # If parking, there is a probability p of moving to the next space
            transitions[(s, "P", s+1)] = p
            # If parking, there is a probability of 1-p of moving to space 52 ( terminal)
            transitions[(s, "P", 52)] = 1 - p
        if s == 51:
            # If in space 51, there is a 100% chance of moving to space 52 (terminal)
            transitions[(s, "P", 52)] = 1
        if s == 45:
            # If in space 45, there is a 100% chance of moving to space 52 (terminal)
            transitions[(s, "C", s+1)] = 1
            transitions[(s, "P", 52)] = 1
    return transitions

# Define rewards for each state-action pair
def get_rewards():
    """
    Define rewards for each state-action pair.
    """
    rewards = {}
    for s in states:
        if s < 50:
            # If not at the end of the parking lot, there is a penalty of -2 for each step
            rewards[(s, "C", s+1)] = -2
            rewards[(s, "P", s+1)] = -2
            # If parking, the reward is based on the distance to the restaurant
            rewards[(s, "P", 52)] = -8*(50-s)
        if s == 50:
            # If in space 50, there is a penalty for parking special lot and then walking to the restaurant
            rewards[(s, "P", 52)] = -8*(50-s)
            rewards[(s, "C", s+1)] = -30
            
        if is_terminal(s):
            # If in the special lot, there is no additional reward or penalty
            rewards[(s, "P", 52)] = 0
            
    return rewards

# Initialize transition probabilities and rewards
transition_probs = get_transition_probabilities()
rewards = get_rewards()

# Initialize the value function
V = {s: 0 for s in states}

# Policy Evaluation
def policy_evaluation(policy):
    """
    Evaluate the given policy to find the value function V.
    """
    gamma = 1
    SMALL_ENOUGH = 1e-3
    it = 0
    while True:
        biggest_change = 0
        for s in states:
            if s in [51, 52]:
                V[s] = 0
            else:
                old_v = V[s]
                new_v = 0
                a = policy[s]
                for s2 in states:
                    r = rewards.get((s, a, s2), 0)
                    new_v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])
                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        print("iter:", it, "biggest_change:", biggest_change)
        it += 1
        if biggest_change < SMALL_ENOUGH:
            break
    return V

# Initialize a random policy
policy = {s: np.random.choice(["P", "C"]) if s not in [51, 52] else "P" for s in states}

# Policy Iteration
def run_policy_iteration():
    """
    Run the policy iteration algorithm to find the optimal policy.
    """
    while True:
        gamma = 1
        V = policy_evaluation(policy)
        is_policy_converged = True
        for s in states:
            if s not in [51, 52]:
                old_a = policy[s]
                best_value = float('-inf')
                for a in action_space:
                    v = 0
                    for s2 in states:
                        r = rewards.get((s, a, s2), 0)
                        v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])
                    if v > best_value:
                        best_value = v
                        policy[s] = a
                if policy[s] != old_a:
                    is_policy_converged = False
        if is_policy_converged:
            break

# Run the policy iteration algorithm
run_policy_iteration()


iter: 0 biggest_change: 154.8
iter: 1 biggest_change: 154.8
iter: 2 biggest_change: 142.0
iter: 3 biggest_change: 87.60000000000001
iter: 4 biggest_change: 58.80000000000001
iter: 5 biggest_change: 51.12000000000003
iter: 6 biggest_change: 46.80000000000001
iter: 7 biggest_change: 28.080000000000013
iter: 8 biggest_change: 21.168000000000006
iter: 9 biggest_change: 14.832000000000008
iter: 10 biggest_change: 14.832000000000008
iter: 11 biggest_change: 12.700800000000015
iter: 12 biggest_change: 12.700800000000015
iter: 13 biggest_change: 7.620480000000015
iter: 14 biggest_change: 4.572288000000015
iter: 15 biggest_change: 3.203711999999996
iter: 16 biggest_change: 3.203711999999996
iter: 17 biggest_change: 3.203711999999996
iter: 18 biggest_change: 1.9222271999999805
iter: 19 biggest_change: 1.153336319999994
iter: 20 biggest_change: 1.153336319999994
iter: 21 biggest_change: 1.153336319999994
iter: 22 biggest_change: 0.692001791999985
iter: 23 biggest_change: 0.692001791999985
iter: 2

In [7]:
rewards

{(1, 'C', 2): -2,
 (1, 'P', 2): -2,
 (1, 'P', 52): -392,
 (2, 'C', 3): -2,
 (2, 'P', 3): -2,
 (2, 'P', 52): -384,
 (3, 'C', 4): -2,
 (3, 'P', 4): -2,
 (3, 'P', 52): -376,
 (4, 'C', 5): -2,
 (4, 'P', 5): -2,
 (4, 'P', 52): -368,
 (5, 'C', 6): -2,
 (5, 'P', 6): -2,
 (5, 'P', 52): -360,
 (6, 'C', 7): -2,
 (6, 'P', 7): -2,
 (6, 'P', 52): -352,
 (7, 'C', 8): -2,
 (7, 'P', 8): -2,
 (7, 'P', 52): -344,
 (8, 'C', 9): -2,
 (8, 'P', 9): -2,
 (8, 'P', 52): -336,
 (9, 'C', 10): -2,
 (9, 'P', 10): -2,
 (9, 'P', 52): -328,
 (10, 'C', 11): -2,
 (10, 'P', 11): -2,
 (10, 'P', 52): -320,
 (11, 'C', 12): -2,
 (11, 'P', 12): -2,
 (11, 'P', 52): -312,
 (12, 'C', 13): -2,
 (12, 'P', 13): -2,
 (12, 'P', 52): -304,
 (13, 'C', 14): -2,
 (13, 'P', 14): -2,
 (13, 'P', 52): -296,
 (14, 'C', 15): -2,
 (14, 'P', 15): -2,
 (14, 'P', 52): -288,
 (15, 'C', 16): -2,
 (15, 'P', 16): -2,
 (15, 'P', 52): -280,
 (16, 'C', 17): -2,
 (16, 'P', 17): -2,
 (16, 'P', 52): -272,
 (17, 'C', 18): -2,
 (17, 'P', 18): -2,
 (17, 'P', 

## Optimal policy for all states

In [3]:
policy

{1: 'C',
 2: 'C',
 3: 'C',
 4: 'C',
 5: 'C',
 6: 'C',
 7: 'C',
 8: 'C',
 9: 'C',
 10: 'C',
 11: 'C',
 12: 'C',
 13: 'C',
 14: 'C',
 15: 'C',
 16: 'C',
 17: 'C',
 18: 'C',
 19: 'C',
 20: 'C',
 21: 'C',
 22: 'C',
 23: 'C',
 24: 'C',
 25: 'C',
 26: 'C',
 27: 'C',
 28: 'C',
 29: 'C',
 30: 'C',
 31: 'C',
 32: 'C',
 33: 'C',
 34: 'C',
 35: 'C',
 36: 'C',
 37: 'C',
 38: 'C',
 39: 'C',
 40: 'C',
 41: 'C',
 42: 'C',
 43: 'C',
 44: 'C',
 45: 'C',
 46: 'C',
 47: 'C',
 48: 'C',
 49: 'C',
 50: 'P',
 51: 'P',
 52: 'P'}

## Optimal values for all states

In [4]:
V

{1: -98.0,
 2: -96.0,
 3: -94.0,
 4: -92.0,
 5: -90.0,
 6: -88.0,
 7: -86.0,
 8: -84.0,
 9: -82.0,
 10: -80.0,
 11: -78.0,
 12: -76.0,
 13: -74.0,
 14: -72.0,
 15: -70.0,
 16: -68.0,
 17: -66.0,
 18: -64.0,
 19: -62.0,
 20: -60.0,
 21: -58.0,
 22: -56.0,
 23: -54.0,
 24: -52.0,
 25: -50.0,
 26: -48.0,
 27: -46.0,
 28: -44.0,
 29: -42.0,
 30: -40.0,
 31: -38.0,
 32: -36.0,
 33: -34.0,
 34: -32.0,
 35: -30.0,
 36: -28.0,
 37: -26.0,
 38: -24.0,
 39: -22.0,
 40: -20.0,
 41: -18.0,
 42: -16.0,
 43: -14.0,
 44: -12.0,
 45: -10.0,
 46: -8.0,
 47: -6.0,
 48: -4.0,
 49: -2.0,
 50: 0.0,
 51: 0,
 52: 0}

# Optimal policy for parking spaces 46 through 50

In [5]:
sorted(policy.items())[45:50]

[(46, 'C'), (47, 'C'), (48, 'C'), (49, 'C'), (50, 'P')]

# Optimal value for parking spaces 46 through 50

In [6]:
sorted(V.items())[44:50]

[(45, -10.0), (46, -8.0), (47, -6.0), (48, -4.0), (49, -2.0), (50, 0.0)]